In [4]:
import nltk
import regex as re
import pandas as pd
from sklearn.utils import shuffle
from nltk import LancasterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import movie_reviews, stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'regex'

In [ ]:
nltk.download('all')

## Data preparation

In [ ]:
data = pd.DataFrame(columns=['text', 'sentiment'])

for id in movie_reviews.fileids():
  text = ' '.join(movie_reviews.words(id))
  sentiment = 1 if movie_reviews.categories(id) == 'pos' else 0
  data = data.append(pd.DataFrame({'text': text,'sentiment': sentiment}, index=[0]))

## Data cleaning

In [ ]:
en_stopwords = list(set(nltk.corpus.stopwords.words('english')))

# remove punctuation from data
clean = [re.sub(r'[^\w\s]','',i).lower() for i in data]

tokens = [word_tokenize(x) for x in data['text']]
filtered_tokens = []

# tokens that are not stopwords collected here
for i in tokens:
  filtered_tokens.append([])
  for j in i:
    if j in en_stopwords:
      continue
    else: filtered_tokens[-1].append(j)

# initialize Lancaster Stemmer
LS = LancasterStemmer()
lemmatized = []
for l in filtered_tokens: lemmatized.append([LS.stem(w) for w in l])

In [ ]:
## Data analysis

## Data analysis

In [ ]:

from operator import itemgetter
from collections import Counter

flat_list = [i for sublist in filtered_tokens for i in sublist]

# Count how many times each word appears
count = Counter(flat_list).items()
sorted_count = sorted(count, key = itemgetter(1))
sorted_count.reverse()

# Select 5000 most frequent words
top5000 = [i[0] for i in sorted_count[:5000]]

## Extracting sentiments for the words

In [ ]:
word_matrix = []

for i in lemmatized: word_matrix.append([1 if j in i else 0 for j in top5000])
features = pd.DataFrame(word_matrix, columns = top5000, index = pd.DataFrame(filtered_tokens))
features['sentiment'] = data['sentiment'].values

In [ ]:
pos_reviews = data[data['sentiment'] == 1]
neg_reviews = data[data['sentiment'] == 0]

## Traing the model for the sentiments using Naive Bayers classifier

In [ ]:
train, test = train_test_split(data, test_size = 0.3)

In [ ]:
cols = train.columns[:-1]

gnb = MultinomialNB()
gnb.fit(train[cols], train['sentiment'])